In [1]:
import pandas as pd
import numpy as np
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import *
from sklearn.metrics import *
import random

In [2]:
df = pd.read_csv("train.csv", header=0, delimiter=",",quoting=10)
dftest = pd.read_csv("test.csv", header=0, delimiter=",",quoting=10)

In [3]:
X = df[["RESOURCE", "MGR_ID", "ROLE_ROLLUP_1", "ROLE_ROLLUP_2", 
                  "ROLE_DEPTNAME","ROLE_TITLE", "ROLE_FAMILY_DESC", "ROLE_FAMILY"]]
X_test = dftest[["RESOURCE", "MGR_ID", "ROLE_ROLLUP_1", "ROLE_ROLLUP_2", 
                  "ROLE_DEPTNAME","ROLE_TITLE", "ROLE_FAMILY_DESC", "ROLE_FAMILY"]]
y= df[['ACTION']]
y = y.as_matrix()
y = np.reshape(y,[32769L,])

In [4]:
#Create classifier
clf=RandomForestClassifier()

In [5]:
#Parameters for clf
estimators = range(200,600)
min_split = range(2,21)
warm = [True, False]
min_leaf = range(1,11)
crit = ['entropy', 'gini']
boot = [True, False] 

In [6]:
param_dist = dict(n_estimators=estimators, 
                  min_samples_split=min_split,
                 warm_start = warm, 
                 min_samples_leaf=min_leaf,
                 criterion=crit,
                 bootstrap=boot)

In [7]:
#Find best parameters
rand=RandomizedSearchCV(clf, param_dist, cv=10, scoring='roc_auc', n_iter=20, 
                        random_state=5, n_jobs=-1)
rand.fit(X,y)
rand.grid_scores_

[mean: 0.85592, std: 0.01389, params: {'warm_start': False, 'bootstrap': True, 'min_samples_leaf': 10, 'n_estimators': 313, 'criterion': 'gini', 'min_samples_split': 13},
 mean: 0.86894, std: 0.01205, params: {'warm_start': True, 'bootstrap': True, 'min_samples_leaf': 2, 'n_estimators': 319, 'criterion': 'entropy', 'min_samples_split': 6},
 mean: 0.86602, std: 0.01186, params: {'warm_start': False, 'bootstrap': False, 'min_samples_leaf': 6, 'n_estimators': 575, 'criterion': 'gini', 'min_samples_split': 17},
 mean: 0.85929, std: 0.01249, params: {'warm_start': False, 'bootstrap': True, 'min_samples_leaf': 9, 'n_estimators': 502, 'criterion': 'entropy', 'min_samples_split': 5},
 mean: 0.86551, std: 0.01233, params: {'warm_start': True, 'bootstrap': False, 'min_samples_leaf': 7, 'n_estimators': 444, 'criterion': 'gini', 'min_samples_split': 15},
 mean: 0.86944, std: 0.01439, params: {'warm_start': True, 'bootstrap': True, 'min_samples_leaf': 3, 'n_estimators': 375, 'criterion': 'entropy',

In [8]:
print rand.best_score_
print rand.best_params_

0.870465255137
{'warm_start': False, 'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 574, 'criterion': 'gini', 'min_samples_split': 15}


In [14]:
# Create and train classifier
clf=RandomForestClassifier(warm_start = False, bootstrap = False,
                           min_samples_leaf = 1, n_estimators = 574,
                           criterion = 'gini', min_samples_split = 15)
clf.fit(X,y)

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=574, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [15]:
#Predict on test set
predictions=clf.predict_proba(X_test)

In [16]:
#Create a submissionfile
def create_submissionfile(predictions, filename):
    """Given a vector of predictions, save results in CSV format."""
    with open(filename, 'w') as f:
        f.write("id,ACTION\n")
        for i, pred in enumerate(predictions):
            f.write("%d,%f\n" % (i + 1, pred))

In [17]:
predictions=predictions[:,1]
filename = raw_input("Enter name for submission file: ")
create_submissionfile(predictions, filename + ".csv")

Enter name for submission file: RF3
